In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
tqdm.pandas()

In [2]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [7]:
print("Shape of Train: ", train.shape)
train.head()

Shape of Train:  (450000, 20)


,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [8]:
train['cancelled'].value_counts()

0    444782
1      5218
Name: cancelled, dtype: int64

Feature Ideas:
- Time to Allot Order to Rider
- Time taken by rider to accept order
- Time for Food Preparation
- Time taken for delivery from time of order
- Check if there is an overlap between rider ID in train and test, if there is, add a column for rider cancellation rate
- If clustering happens for time of day of cancellation, add as a feature
- Ratio of first mile to last mile distance

Cleaning Ideas:
- lifetime_order_count change as per date
- interpolate based on rider_id for session time

Future Ideas:
- Subsample first and then smote in order to prevent smote from messing up

In [9]:
train[train['accept_time'].isna()].head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
1700,2021-01-26 06:35:16,558453,2021-01-26 00:00:00,2021-01-26 06:35:18,NaN,NaN,NaN,15903,1.8026,8.55,50.0,50.0,1,0.0,1216.0,NaN,NaN,NaN,51.300000,2021-01-26 06:40:17
4066,2021-01-26 07:20:44,560819,2021-01-26 00:00:00,2021-01-26 07:20:45,NaN,NaN,NaN,3430,1.6112,4.29,29.0,29.0,1,0.0,190.0,NaN,NaN,NaN,52.700000,2021-01-26 07:21:20
9296,2021-01-26 08:58:12,566050,2021-01-26 00:00:00,2021-01-26 08:58:41,NaN,NaN,NaN,20621,2.8689,4.07,NaN,NaN,1,NaN,917.0,NaN,NaN,NaN,120.733333,2021-01-26 09:00:35
13365,2021-01-26 12:42:36,570118,2021-01-26 00:00:00,2021-01-26 12:42:37,NaN,NaN,NaN,10605,0.7966,3.14,11.0,11.0,1,0.0,1546.0,NaN,NaN,NaN,478.333333,2021-01-26 12:47:37
13429,2021-01-26 12:45:42,570182,2021-01-26 00:00:00,2021-01-26 12:45:43,NaN,NaN,NaN,3585,0.9797,7.79,67.0,65.0,1,2.0,144.0,NaN,NaN,NaN,325.666667,2021-01-26 12:47:38


In [10]:
print("Shape of Test: ", test.shape)
test.head()

Shape of Test:  (144844, 16)


,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.6585,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.0709,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.3884,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.9039,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.8275,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333


In [11]:
len(set(train['rider_id']).intersection(set(test['rider_id'])))

11652

In [12]:
len(test['rider_id'].unique())

13682

In [13]:
train[train['rider_id'] == 1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
315699,2021-02-03 06:43:10,239802,2021-02-03 00:00:00,2021-02-03 06:43:25,2021-02-03 06:43:30,2021-02-03 06:55:20,2021-02-03 07:12:18,1,1.4561,4.14,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,147.533333,NaN
328569,2021-02-03 13:38:38,252675,2021-02-03 00:00:00,2021-02-03 13:38:55,2021-02-03 13:40:17,2021-02-03 14:06:26,2021-02-03 14:46:21,1,2.4127,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,510.566667,NaN
331400,2021-02-03 14:02:48,255506,2021-02-03 00:00:00,2021-02-03 14:04:44,2021-02-03 14:05:50,2021-02-03 14:15:04,2021-02-03 14:33:03,1,0.8029,3.68,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,534.733333,NaN
346038,2021-02-03 16:14:20,270142,2021-02-03 00:00:00,2021-02-03 16:14:55,2021-02-03 16:15:50,2021-02-03 16:31:13,2021-02-03 16:40:45,1,1.1215,3.58,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,666.266667,NaN
348876,2021-02-03 16:41:11,272980,2021-02-03 00:00:00,2021-02-03 16:41:47,2021-02-03 16:42:43,2021-02-03 16:49:22,2021-02-03 16:58:00,1,0.4628,1.51,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,693.116667,NaN
352353,2021-02-03 17:14:48,276457,2021-02-03 00:00:00,2021-02-03 17:15:39,2021-02-03 17:16:26,2021-02-03 17:28:24,2021-02-03 17:35:16,1,1.1628,1.36,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,726.733333,NaN
353553,2021-02-03 17:26:31,277657,2021-02-03 00:00:00,2021-02-03 17:26:55,2021-02-03 17:27:51,2021-02-03 17:41:59,2021-02-03 18:04:50,1,1.0028,3.59,6.0,6.0,0,0.0,1380.0,NaN,NaN,NaN,738.450000,NaN
373828,2021-02-04 14:14:41,216800,2021-02-04 00:00:00,2021-02-04 14:15:41,2021-02-04 14:16:31,2021-02-04 14:29:13,2021-02-04 14:41:52,1,0.8283,1.91,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,519.200000,NaN
379722,2021-02-04 15:10:50,222695,2021-02-04 00:00:00,2021-02-04 15:11:47,2021-02-04 15:12:24,2021-02-04 15:15:29,2021-02-04 15:23:42,1,0.2979,0.58,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,575.350000,NaN
383660,2021-02-04 15:48:53,226632,2021-02-04 00:00:00,2021-02-04 15:49:39,2021-02-04 15:49:52,2021-02-04 15:55:55,2021-02-04 16:04:07,1,0.8439,0.62,12.0,12.0,0,0.0,1380.0,NaN,NaN,NaN,613.400000,NaN


In [14]:
test[test['rider_id'] == 1].head(10)

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
5451,2021-02-06 13:42:02,135683,2021-02-06 00:00:00,2021-02-06 13:42:48,2021-02-06 13:44:05,1,0.669800,4.93,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,600.333333
17324,2021-02-06 16:02:40,147554,2021-02-06 00:00:00,2021-02-06 16:02:59,2021-02-06 16:03:42,1,1.324600,0.54,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,740.966667
18092,2021-02-06 16:11:36,148323,2021-02-06 00:00:00,2021-02-06 16:11:52,2021-02-06 16:13:21,1,0.029638,0.23,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,749.900000
23756,2021-02-06 17:19:38,153986,2021-02-06 00:00:00,2021-02-06 17:22:20,2021-02-06 17:22:24,1,1.311700,4.69,20.0,20.0,0.0,1380.0,NaN,NaN,NaN,817.933333
32469,2021-02-07 08:03:10,86420,2021-02-07 00:00:00,2021-02-07 08:03:10,2021-02-07 08:04:41,1,0.965400,0.19,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,262.700000
41660,2021-02-07 13:33:05,95611,2021-02-07 00:00:00,2021-02-07 13:33:06,2021-02-07 13:33:15,1,2.511500,0.11,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,574.283333
44538,2021-02-07 13:58:36,98489,2021-02-07 00:00:00,2021-02-07 13:58:38,2021-02-07 13:58:50,1,1.551600,3.87,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,599.800000
46696,2021-02-07 14:19:07,100647,2021-02-07 00:00:00,2021-02-07 14:19:08,2021-02-07 14:19:18,1,0.929400,5.24,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,620.316667
50516,2021-02-07 15:21:35,104468,2021-02-07 00:00:00,2021-02-07 15:22:01,2021-02-07 15:23:02,1,0.322100,3.08,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,682.783333
58446,2021-02-07 16:47:26,112397,2021-02-07 00:00:00,2021-02-07 16:47:27,2021-02-07 16:47:44,1,1.536600,1.15,24.0,24.0,0.0,1380.0,NaN,NaN,NaN,768.633333


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            450000 non-null  object 
 1   order_id              450000 non-null  int64  
 2   order_date            450000 non-null  object 
 3   allot_time            450000 non-null  object 
 4   accept_time           449843 non-null  object 
 5   pickup_time           447579 non-null  object 
 6   delivered_time        444782 non-null  object 
 7   rider_id              450000 non-null  int64  
 8   first_mile_distance   450000 non-null  float64
 9   last_mile_distance    450000 non-null  float64
 10  alloted_orders        433052 non-null  float64
 11  delivered_orders      432659 non-null  float64
 12  cancelled             450000 non-null  int64  
 13  undelivered_orders    432659 non-null  float64
 14  lifetime_order_count  449947 non-null  float64
 15  

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144844 entries, 0 to 144843
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            144844 non-null  object 
 1   order_id              144844 non-null  int64  
 2   order_date            144844 non-null  object 
 3   allot_time            144844 non-null  object 
 4   accept_time           144776 non-null  object 
 5   rider_id              144844 non-null  int64  
 6   first_mile_distance   144844 non-null  float64
 7   last_mile_distance    144844 non-null  float64
 8   alloted_orders        140071 non-null  float64
 9   delivered_orders      139960 non-null  float64
 10  undelivered_orders    139960 non-null  float64
 11  lifetime_order_count  144066 non-null  float64
 12  reassignment_method   4632 non-null    object 
 13  reassignment_reason   4635 non-null    object 
 14  reassigned_order      4635 non-null    float64
 15  

In [3]:
drop_cols = ['pickup_time', 'delivered_time', 'cancelled_time']
train.drop(drop_cols, axis = 1, inplace = True)

In [4]:
def time_diff(time_list_1, time_list_2):
    hour_diff = (int(time_list_1[0]) - int(time_list_2[0]))*3600
    min_diff = (int(time_list_1[1]) - int(time_list_2[1]))*60
    sec_diff = (int(time_list_1[2]) - int(time_list_2[2]))
    return hour_diff + min_diff + sec_diff
    
def create_time_features(row):
    
    order_times = row['order_time'].split(' ')[1].split(':')
    allot_times = row['allot_time'].split(' ')[1].split(':')
    if row['accept_time'] == row['accept_time']:
        accept_times = row['accept_time'].split(' ')[1].split(':')
    
    row['rider_allot_time'] = time_diff(allot_times, order_times)
    try:
        row['rider_accept_time'] = time_diff(accept_times, allot_times)
    except:
        row['rider_accept_time'] = -1
        
    return row

In [15]:
def apply_func(df):
    df = df.progress_apply(create_time_features, axis = 1)
    return df

def parallelize_dataframe(df, func, n_cores = 10):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def interp(x):
    d = pd.DataFrame(x['session_time'].values, index = pd.to_datetime(x['order_time']), columns = ['session_time'])
    d = d.interpolate('time')
    d = d.interpolate('bfill')
    
    x['session_time'] = d['session_time'].values
    return x

In [6]:
grp_obj = train.groupby(['order_date', 'rider_id'])
frames = []

for name, grp in tqdm(grp_obj):
    grp = interp(grp)
    frames+=[grp]
    
train = pd.concat(frames)
train.sort_index(inplace = True)

100%|████████████████████████████████████████████████████████████████████████████| 94620/94620 [02:40<00:00, 590.01it/s]


In [7]:
grp_obj = test.groupby(['order_date', 'rider_id'])
frames = []

for name, grp in tqdm(grp_obj):
    grp = interp(grp)
    frames+=[grp]
    
test = pd.concat(frames)
test.sort_index(inplace = True)

100%|████████████████████████████████████████████████████████████████████████████| 31680/31680 [00:55<00:00, 566.61it/s]


In [8]:
train = parallelize_dataframe(train, apply_func)
test = parallelize_dataframe(test, apply_func)

100%|████████████████████████████████████████████████████████████████████████████| 14485/14485 [00:30<00:00, 477.69it/s]


In [44]:
train['ratio'] = train['first_mile_distance']/(train['last_mile_distance'] + 1e-7)
test['ratio'] = test['first_mile_distance']/(test['last_mile_distance'] + 1e-7)

In [10]:
more_drop = ['order_time', 'order_id', 'order_date', 'allot_time',
             'accept_time', 'rider_id']

train.drop(more_drop, axis = 1, inplace = True)
test.drop(more_drop, axis = 1, inplace = True)

In [21]:
from sklearn.preprocessing import LabelEncoder
text_fill = ['reassignment_method', 'reassignment_reason']
num_fill = ['reassigned_order', 'session_time', 'alloted_orders',
       'delivered_orders', 'undelivered_orders', 'lifetime_order_count',]

for col in text_fill:
    le = LabelEncoder()
    train[col] = train[col].fillna('Missing')
    train[col] = le.fit_transform(train[col])
    test[col] = test[col].fillna('Missing')
    test[col] = le.transform(test[col])

for col in num_fill:
    train[col] = train[col].fillna(-1)
    test[col] = test[col].fillna(-1)

In [47]:
sc = MinMaxScaler()
sc = sc.fit(train['ratio'].values.reshape(-1,1))
train['ratio'] = sc.transform(train['ratio'].values.reshape(-1,1))

In [49]:
train['reassigned_order'].value_counts()

-1.0    436247
 1.0     13753
Name: reassigned_order, dtype: int64

In [57]:
X_data = train.drop('cancelled', axis = 1)
y_data = train['cancelled']

kfold = StratifiedKFold(random_state = 10, n_splits = 10, shuffle = True)
splits = kfold.split(X_data, y_data)

sm = SMOTE(random_state = 10, k_neighbors = 7, n_jobs = -1, sampling_strategy = 1/10)

roc_scores = []
Models = []

for Train, Test in splits:
    X_Train, X_Val, Y_Train, Y_Val = X_data.iloc[Train], X_data.iloc[Test], y_data.iloc[Train], y_data.iloc[Test]
    xgb = XGBClassifier(n_estimators = 2000, max_depth = 8, n_jobs = -1,
                        learning_rate = 0.05, reg_lambda = 0.1, random_state = 10)
    X_Train, Y_Train = sm.fit_resample(X_Train, Y_Train)
    xgb.fit(X_Train, Y_Train, early_stopping_rounds = 50,
            eval_set = [(X_Train, Y_Train), (X_Val, Y_Val)],
            eval_metric = 'logloss', verbose = True)
    pred = xgb.predict(X_Val)
    
    roc_scores += [roc_auc_score(pred, Y_Val)]
    print("ROC-AUC Score: ", np.mean(roc_scores))
    Models.append(xgb)

/home/adasditya/.local/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-logloss:0.65106	validation_1-logloss:0.64895
[1]	validation_0-logloss:0.61323	validation_1-logloss:0.60887
[2]	validation_0-logloss:0.57868	validation_1-logloss:0.57232
[3]	validation_0-logloss:0.54714	validation_1-logloss:0.53892
[4]	validation_0-logloss:0.51838	validation_1-logloss:0.50828
[5]	validation_0-logloss:0.49206	validation_1-logloss:0.48012
[6]	validation_0-logloss:0.46775	validation_1-logloss:0.45410
[7]	validation_0-logloss:0.44533	validation_1-logloss:0.43005
[8]	validation_0-logloss:0.42462	validation_1-logloss:0.40778
[9]	validation_0-logloss:0.40551	validation_1-logloss:0.38712
[10]	validation_0-logloss:0.38777	validation_1-logloss:0.36794
[11]	validation_0-logloss:0.37138	validation_1-logloss:0.35006
[12]	validation_0-logloss:0.35611	validation_1-logloss:0.33338
[13]	validation_0-logloss:0.34191	validation_1-logloss:0.31783
[14]	validation_0-logloss:0.32878	validation_1-logloss:0.30331
[15]	validation_0-logloss:0.31615	validation_1-logloss:0.28975
[1

[130]	validation_0-logloss:0.10619	validation_1-logloss:0.06523
[131]	validation_0-logloss:0.10598	validation_1-logloss:0.06517
[132]	validation_0-logloss:0.10570	validation_1-logloss:0.06510
[133]	validation_0-logloss:0.10546	validation_1-logloss:0.06502
[134]	validation_0-logloss:0.10518	validation_1-logloss:0.06496
[135]	validation_0-logloss:0.10488	validation_1-logloss:0.06486
[136]	validation_0-logloss:0.10479	validation_1-logloss:0.06483
[137]	validation_0-logloss:0.10454	validation_1-logloss:0.06476
[138]	validation_0-logloss:0.10435	validation_1-logloss:0.06471
[139]	validation_0-logloss:0.10418	validation_1-logloss:0.06467
[140]	validation_0-logloss:0.10407	validation_1-logloss:0.06465
[141]	validation_0-logloss:0.10381	validation_1-logloss:0.06459
[142]	validation_0-logloss:0.10370	validation_1-logloss:0.06455
[143]	validation_0-logloss:0.10355	validation_1-logloss:0.06451
[144]	validation_0-logloss:0.10337	validation_1-logloss:0.06445
[145]	validation_0-logloss:0.10320	valid

[259]	validation_0-logloss:0.08946	validation_1-logloss:0.06200
[260]	validation_0-logloss:0.08940	validation_1-logloss:0.06200
[261]	validation_0-logloss:0.08932	validation_1-logloss:0.06199
[262]	validation_0-logloss:0.08926	validation_1-logloss:0.06198
[263]	validation_0-logloss:0.08915	validation_1-logloss:0.06197
[264]	validation_0-logloss:0.08910	validation_1-logloss:0.06195
[265]	validation_0-logloss:0.08892	validation_1-logloss:0.06193
[266]	validation_0-logloss:0.08890	validation_1-logloss:0.06193
[267]	validation_0-logloss:0.08881	validation_1-logloss:0.06192
[268]	validation_0-logloss:0.08870	validation_1-logloss:0.06191
[269]	validation_0-logloss:0.08865	validation_1-logloss:0.06190
[270]	validation_0-logloss:0.08852	validation_1-logloss:0.06190
[271]	validation_0-logloss:0.08838	validation_1-logloss:0.06188
[272]	validation_0-logloss:0.08826	validation_1-logloss:0.06186
[273]	validation_0-logloss:0.08818	validation_1-logloss:0.06183
[274]	validation_0-logloss:0.08809	valid

[388]	validation_0-logloss:0.07903	validation_1-logloss:0.06073
[389]	validation_0-logloss:0.07898	validation_1-logloss:0.06073
[390]	validation_0-logloss:0.07895	validation_1-logloss:0.06073
[391]	validation_0-logloss:0.07890	validation_1-logloss:0.06072
[392]	validation_0-logloss:0.07884	validation_1-logloss:0.06071
[393]	validation_0-logloss:0.07881	validation_1-logloss:0.06071
[394]	validation_0-logloss:0.07870	validation_1-logloss:0.06070
[395]	validation_0-logloss:0.07864	validation_1-logloss:0.06069
[396]	validation_0-logloss:0.07858	validation_1-logloss:0.06069
[397]	validation_0-logloss:0.07855	validation_1-logloss:0.06068
[398]	validation_0-logloss:0.07848	validation_1-logloss:0.06067
[399]	validation_0-logloss:0.07842	validation_1-logloss:0.06066
[400]	validation_0-logloss:0.07839	validation_1-logloss:0.06067
[401]	validation_0-logloss:0.07826	validation_1-logloss:0.06067
[402]	validation_0-logloss:0.07817	validation_1-logloss:0.06067
[403]	validation_0-logloss:0.07806	valid

...

[1032]	validation_0-logloss:0.04884	validation_1-logloss:0.05796
[1033]	validation_0-logloss:0.04875	validation_1-logloss:0.05795
[1034]	validation_0-logloss:0.04873	validation_1-logloss:0.05795
[1035]	validation_0-logloss:0.04868	validation_1-logloss:0.05794
[1036]	validation_0-logloss:0.04867	validation_1-logloss:0.05794
[1037]	validation_0-logloss:0.04864	validation_1-logloss:0.05794
[1038]	validation_0-logloss:0.04863	validation_1-logloss:0.05794
[1039]	validation_0-logloss:0.04860	validation_1-logloss:0.05793
[1040]	validation_0-logloss:0.04858	validation_1-logloss:0.05793
[1041]	validation_0-logloss:0.04856	validation_1-logloss:0.05792
[1042]	validation_0-logloss:0.04855	validation_1-logloss:0.05793
[1043]	validation_0-logloss:0.04850	validation_1-logloss:0.05793
[1044]	validation_0-logloss:0.04848	validation_1-logloss:0.05793
[1045]	validation_0-logloss:0.04844	validation_1-logloss:0.05792
[1046]	validation_0-logloss:0.04843	validation_1-logloss:0.05793
[1047]	validation_0-loglo

[1159]	validation_0-logloss:0.04517	validation_1-logloss:0.05768
[1160]	validation_0-logloss:0.04513	validation_1-logloss:0.05767
[1161]	validation_0-logloss:0.04510	validation_1-logloss:0.05768
[1162]	validation_0-logloss:0.04510	validation_1-logloss:0.05768
[1163]	validation_0-logloss:0.04506	validation_1-logloss:0.05767
[1164]	validation_0-logloss:0.04505	validation_1-logloss:0.05767
[1165]	validation_0-logloss:0.04504	validation_1-logloss:0.05767
[1166]	validation_0-logloss:0.04503	validation_1-logloss:0.05767
[1167]	validation_0-logloss:0.04502	validation_1-logloss:0.05767
[1168]	validation_0-logloss:0.04500	validation_1-logloss:0.05767
[1169]	validation_0-logloss:0.04496	validation_1-logloss:0.05768
[1170]	validation_0-logloss:0.04485	validation_1-logloss:0.05765
[1171]	validation_0-logloss:0.04484	validation_1-logloss:0.05765
[1172]	validation_0-logloss:0.04482	validation_1-logloss:0.05766
[1173]	validation_0-logloss:0.04474	validation_1-logloss:0.05765
[1174]	validation_0-loglo

[1286]	validation_0-logloss:0.04151	validation_1-logloss:0.05729
[1287]	validation_0-logloss:0.04149	validation_1-logloss:0.05728
[1288]	validation_0-logloss:0.04146	validation_1-logloss:0.05727
[1289]	validation_0-logloss:0.04144	validation_1-logloss:0.05727
[1290]	validation_0-logloss:0.04139	validation_1-logloss:0.05727
[1291]	validation_0-logloss:0.04134	validation_1-logloss:0.05726
[1292]	validation_0-logloss:0.04133	validation_1-logloss:0.05726
[1293]	validation_0-logloss:0.04131	validation_1-logloss:0.05726
[1294]	validation_0-logloss:0.04128	validation_1-logloss:0.05726
[1295]	validation_0-logloss:0.04127	validation_1-logloss:0.05726
[1296]	validation_0-logloss:0.04126	validation_1-logloss:0.05727
[1297]	validation_0-logloss:0.04125	validation_1-logloss:0.05727
[1298]	validation_0-logloss:0.04124	validation_1-logloss:0.05727
[1299]	validation_0-logloss:0.04124	validation_1-logloss:0.05728
[1300]	validation_0-logloss:0.04124	validation_1-logloss:0.05728
[1301]	validation_0-loglo

[1413]	validation_0-logloss:0.03852	validation_1-logloss:0.05702
[1414]	validation_0-logloss:0.03847	validation_1-logloss:0.05701
[1415]	validation_0-logloss:0.03843	validation_1-logloss:0.05700
[1416]	validation_0-logloss:0.03841	validation_1-logloss:0.05699
[1417]	validation_0-logloss:0.03837	validation_1-logloss:0.05698
[1418]	validation_0-logloss:0.03833	validation_1-logloss:0.05697
[1419]	validation_0-logloss:0.03830	validation_1-logloss:0.05697
[1420]	validation_0-logloss:0.03828	validation_1-logloss:0.05697
[1421]	validation_0-logloss:0.03826	validation_1-logloss:0.05697
[1422]	validation_0-logloss:0.03823	validation_1-logloss:0.05696
[1423]	validation_0-logloss:0.03822	validation_1-logloss:0.05696
[1424]	validation_0-logloss:0.03819	validation_1-logloss:0.05696
[1425]	validation_0-logloss:0.03817	validation_1-logloss:0.05695
[1426]	validation_0-logloss:0.03816	validation_1-logloss:0.05695
[1427]	validation_0-logloss:0.03814	validation_1-logloss:0.05696
[1428]	validation_0-loglo

[1540]	validation_0-logloss:0.03604	validation_1-logloss:0.05686
[1541]	validation_0-logloss:0.03602	validation_1-logloss:0.05686
[1542]	validation_0-logloss:0.03599	validation_1-logloss:0.05686
[1543]	validation_0-logloss:0.03597	validation_1-logloss:0.05686
[1544]	validation_0-logloss:0.03596	validation_1-logloss:0.05686
[1545]	validation_0-logloss:0.03596	validation_1-logloss:0.05686
[1546]	validation_0-logloss:0.03594	validation_1-logloss:0.05686
[1547]	validation_0-logloss:0.03591	validation_1-logloss:0.05685
[1548]	validation_0-logloss:0.03590	validation_1-logloss:0.05685
[1549]	validation_0-logloss:0.03588	validation_1-logloss:0.05685
[1550]	validation_0-logloss:0.03587	validation_1-logloss:0.05685
[1551]	validation_0-logloss:0.03584	validation_1-logloss:0.05685
[1552]	validation_0-logloss:0.03583	validation_1-logloss:0.05685
[1553]	validation_0-logloss:0.03582	validation_1-logloss:0.05686
[1554]	validation_0-logloss:0.03581	validation_1-logloss:0.05686
[1555]	validation_0-loglo